In [1]:
import os
import sys

os.environ["SPARK_HOME"] = "/usr/spark2.4.3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
conf = SparkConf().setAppName("appName")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
df = spark.read.format("csv").option("inferSchema", True).option("header", True)\
.load("file:/home/wilsonsagar8680/tinku/by-day/2010-12-01.csv")

df.printSchema()
df.createOrReplaceTempView("dftable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [4]:
df.limit(5).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+



In [5]:
from pyspark.sql.functions import col

df.select("InvoiceNo", "Description").where(col("InvoiceNo") == 536365).show(5, False)

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
+---------+-----------------------------------+
only showing top 5 rows



In [6]:
df.where("InvoiceNo == 536365")\
.show(50, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |22752

In [7]:
descripFilter = df.Description.contains("POSTAGE")

In [8]:
priceFilter = col("UnitPrice") > 600
#cidFilter = df.CustomerID.isNotNull()
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [9]:
from pyspark.sql.functions import expr

df.withColumn("isExpensive", col("UnitPrice") > 250).where("isExpensive")\
.select("Description", "UnitPrice", "isExpensive").show(5)

+--------------+---------+-----------+
|   Description|UnitPrice|isExpensive|
+--------------+---------+-----------+
|DOTCOM POSTAGE|   569.77|       true|
|DOTCOM POSTAGE|   607.49|       true|
+--------------+---------+-----------+



In [10]:
from pyspark.sql.functions import pow

realQ = pow((df.Quantity * df.UnitPrice), 2) + 5
df.select(df.CustomerID,realQ.alias("Real Quantity")).show(5)

+----------+------------------+
|CustomerID|     Real Quantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import round, bround

roundedQ = round(realQ)
df.select(df.CustomerID, roundedQ.alias("Rounded Quantity")).show(5)

+----------+----------------+
|CustomerID|Rounded Quantity|
+----------+----------------+
|   17850.0|           239.0|
|   17850.0|           419.0|
|   17850.0|           489.0|
|   17850.0|           419.0|
|   17850.0|           419.0|
+----------+----------------+
only showing top 5 rows



In [12]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|               0.0|           12431.0|     Australia|
|    max|          C

In [13]:
df.stat.freqItems(["StockCode", "Quantity"]).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



In [14]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description")).alias("Description")).show()

+--------------------+
|         Description|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
|Set 7 Babushka Ne...|
|Glass Star Froste...|
|Hand Warmer Union...|
|Hand Warmer Red P...|
|Assorted Colour B...|
|Poppy's Playhouse...|
|Poppy's Playhouse...|
|Feltcraft Princes...|
|Ivory Knitted Mug...|
|Box Of 6 Assorted...|
|Box Of Vintage Ji...|
|Box Of Vintage Al...|
|Home Building Blo...|
|Love Building Blo...|
|Recipe Box With M...|
+--------------------+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import regexp_replace

target = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(regexp_replace("Description", target, "COLOR").alias("RepColor"), "Description").show(5)

+--------------------+--------------------+
|            RepColor|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
|CREAM CUPID HEART...|CREAM CUPID HEART...|
|KNITTED UNION FLA...|KNITTED UNION FLA...|
|COLOR WOOLLY HOTT...|RED WOOLLY HOTTIE...|
+--------------------+--------------------+
only showing top 5 rows



In [16]:
from pyspark.sql.functions import regexp_extract

extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select( \
regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),\
col("Description")).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



In [17]:
df.withColumn("hasColor", col("Description").contains("WHITE") | col("Description").contains("BLACK"))\
.select("Description", "hasColor").where(col("hasColor")).show(5)

+--------------------+--------+
|         Description|hasColor|
+--------------------+--------+
|WHITE HANGING HEA...|    true|
| WHITE METAL LANTERN|    true|
|RED WOOLLY HOTTIE...|    true|
|WHITE HANGING HEA...|    true|
| WHITE METAL LANTERN|    true|
+--------------------+--------+
only showing top 5 rows



In [18]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [19]:
from pyspark.sql.functions import current_date, current_timestamp
datedf = spark.range(10).withColumn("Today", current_date()).withColumn("Now", current_timestamp())

In [20]:
datedf.printSchema()

root
 |-- id: long (nullable = false)
 |-- Today: date (nullable = false)
 |-- Now: timestamp (nullable = false)



In [21]:
from pyspark.sql.functions import date_add, date_sub

datedf.select(date_sub("Today", 7).alias("Subtracted Date"), date_add("Today", 7).alias("Added Date")).show(5)

+---------------+----------+
|Subtracted Date|Added Date|
+---------------+----------+
|     2020-06-16|2020-06-30|
|     2020-06-16|2020-06-30|
|     2020-06-16|2020-06-30|
|     2020-06-16|2020-06-30|
|     2020-06-16|2020-06-30|
+---------------+----------+
only showing top 5 rows



In [22]:
from pyspark.sql.functions import datediff, months_between, to_date
datedf.withColumn("Week Ago", date_sub(col("Today"), 7)).withColumn("week_ago", date_sub(col("today"), 7))\
.select(datedf.Today, "Week Ago", datediff(col("week_ago"), col("today")).alias("Diff")).show(1)

+----------+----------+----+
|     Today|  Week Ago|Diff|
+----------+----------+----+
|2020-06-23|2020-06-16|  -7|
+----------+----------+----+
only showing top 1 row



In [23]:
datedf.withColumn("Week Ago", date_sub(col("Today"), 7)).withColumn("week_ago", date_sub(col("today"), 7))\
.select(datedf.Today, "Week Ago", datediff(col("today"), col("week_ago")).alias("Diff")).show(1)

+----------+----------+----+
|     Today|  Week Ago|Diff|
+----------+----------+----+
|2020-06-23|2020-06-16|   7|
+----------+----------+----+
only showing top 1 row



In [24]:
from pyspark.sql.functions import lit
datedf.select(
to_date(lit("2016-01-01")).alias("start"),
to_date(lit("2017-05-22")).alias("end"))\
.select(months_between(col("end"),col("start"))).show(1)

+--------------------------------+
|months_between(end, start, true)|
+--------------------------------+
|                     16.67741935|
+--------------------------------+
only showing top 1 row



In [25]:
datedf.select(to_date(lit("2017-20-12")), to_date(lit("2017-12-1"))).show(1)

+---------------------+--------------------+
|to_date('2017-20-12')|to_date('2017-12-1')|
+---------------------+--------------------+
|                 null|          2017-12-01|
+---------------------+--------------------+
only showing top 1 row



In [26]:
from pyspark.sql.functions import to_timestamp
dform = "dd-MM-yyyy"
datedf.select(to_date(lit("12-11-2019"), dform).alias("Normal date"), to_timestamp(lit("12-11-2019"), dform).\
              alias("Normal TS")).show()

+-----------+-------------------+
|Normal date|          Normal TS|
+-----------+-------------------+
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
| 2019-11-12|2019-11-12 00:00:00|
+-----------+-------------------+



In [27]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [28]:
from pyspark.sql.functions import coalesce
df.select(coalesce("Description", "CustomerID")).show()

+---------------------------------+
|coalesce(Description, CustomerID)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



In [29]:
df.count()

3108

In [30]:
df.na.drop("any")
df.count()

3108

In [31]:
nrow = spark.createDataFrame([["Alice", "","","","","","",""]])

In [32]:
newdf = df.union(nrow)

In [33]:
newdf.count()

3109

In [34]:
newdf.na.fill("Bob")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

In [35]:
from pyspark.sql.functions import struct
complexdf = df.select(struct("Description", "InvoiceNo").alias("Complex"))

In [36]:
complexdf.take(2)

[Row(Complex=Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', InvoiceNo='536365')),
 Row(Complex=Row(Description='WHITE METAL LANTERN', InvoiceNo='536365'))]

In [37]:
complexdf.select("Complex.InvoiceNo").show()

+---------+
|InvoiceNo|
+---------+
|   536365|
|   536365|
|   536365|
|   536365|
|   536365|
|   536365|
|   536365|
|   536366|
|   536366|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
+---------+
only showing top 20 rows



In [38]:
distinctdf = complexdf.select("Complex.InvoiceNo").distinct()

In [39]:
distinctdf.show(5)

+---------+
|InvoiceNo|
+---------+
|   536596|
|   536597|
|   536414|
|   536550|
|   536460|
+---------+
only showing top 5 rows



In [40]:
complexdf.distinct().count()

3007

In [41]:
distinctdf.count()

143

In [42]:
from pyspark.sql.functions import split
df.select(split("Description", " ")).show(2)

+---------------------+
|split(Description,  )|
+---------------------+
| [WHITE, HANGING, ...|
| [WHITE, METAL, LA...|
+---------------------+
only showing top 2 rows



In [43]:
from pyspark.sql.functions import size
df.select(size(split("Description", " ")).alias("Size")).show(2)

+----+
|Size|
+----+
|   5|
|   3|
+----+
only showing top 2 rows



In [44]:
from pyspark.sql.functions import array_contains

df.select(array_contains(split("Description", " "), "WHITE")).show(4)

+--------------------------------------------+
|array_contains(split(Description,  ), WHITE)|
+--------------------------------------------+
|                                        true|
|                                        true|
|                                       false|
|                                       false|
+--------------------------------------------+
only showing top 4 rows



In [45]:
from pyspark.sql.functions import create_map
df.select(create_map("InvoiceNo", "Description")).show(3)

+---------------------------+
|map(InvoiceNo, Description)|
+---------------------------+
|       [536365 -> WHITE ...|
|       [536365 -> WHITE ...|
|       [536365 -> CREAM ...|
+---------------------------+
only showing top 3 rows



In [46]:
jsondf = spark.read.option("inferSchema", True).json("file:/home/wilsonsagar8680/tinku/2015-summary.json")

In [47]:
jsondf.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [48]:
from pyspark.sql.functions import *


In [49]:
data = [("1", '''{"f1": "value1", "f2": "value2"}'''), ("2", '''{"f1": "value12"}''')]

In [50]:
data

[('1', '{"f1": "value1", "f2": "value2"}'), ('2', '{"f1": "value12"}')]

In [51]:
data[0]

('1', '{"f1": "value1", "f2": "value2"}')

In [52]:
data[0][1]

'{"f1": "value1", "f2": "value2"}'

In [53]:
jsondf.select(col("DEST_COUNTRY_NAME")).show(3)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
+-----------------+
only showing top 3 rows



In [54]:
mixedf = spark.createDataFrame(data, ("id", "jstr"))

In [55]:
mixedf.select(get_json_object(col("jstr"), "$.f1").alias("Values")).show()

+-------+
| Values|
+-------+
| value1|
|value12|
+-------+



In [56]:
mixedf.select(get_json_object(col("jstr"), "$.f2").alias("f2 values")).collect()

[Row(f2 values='value2'), Row(f2 values=None)]

In [57]:
jsondf.select("*").alias("Si").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [58]:
njdf = jsondf.selectExpr("(DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count) as complex").select(to_json("complex").alias("complex"))

In [59]:
njdf.show(5)
njdf.printSchema()

+--------------------+
|             complex|
+--------------------+
|{"DEST_COUNTRY_NA...|
|{"DEST_COUNTRY_NA...|
|{"DEST_COUNTRY_NA...|
|{"DEST_COUNTRY_NA...|
|{"DEST_COUNTRY_NA...|
+--------------------+
only showing top 5 rows

root
 |-- complex: string (nullable = true)



In [66]:
njdf.select(get_json_object(col("complex"), "$.ORIGIN_COUNTRY_NAME")).alias("Origins").show(5)

+-----------------------------------------------+
|get_json_object(complex, $.ORIGIN_COUNTRY_NAME)|
+-----------------------------------------------+
|                                        Romania|
|                                        Croatia|
|                                        Ireland|
|                                  United States|
|                                          India|
+-----------------------------------------------+
only showing top 5 rows



In [ ]:
njdf.select(to_json("complex"))